# Quantitative Analysis - Fidelity
## Load dataset

In [30]:
from utils.utils import gait_dataloader
from utils.utils import init_model,  patients_idx_to_cycles_idx, find_cycles_idx_by_patient_idx
from sklearn.model_selection import train_test_split

from utils.utils import init_model,accuracy_end_to_end
import os
import numpy as np

from sklearn.svm import SVC

from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix

import seaborn as sn

from sklearn.metrics import f1_score


from utils.utils import *

def load_dataset_v1(dir_dataset,channel_first=True,flatten=True,shuffle=False,random_state=0):
    index = [0,1,2,3,4,5,6,7,8,9,14,15,16,17,18,19,20,21,22,23,24,29]
    
    npzfile = np.load(dir_dataset + 'RightPC.npz')
    
    x_train_all = npzfile['Input']
    y_train = npzfile['Output']
    y_train = np.ravel(y_train)
    x_train = x_train_all[:,:,index]
    if channel_first:
        x_train = x_train.transpose(0,2,1)
    
    npzfile = np.load(dir_dataset + 'LeftPC.npz')
    x_val_all = npzfile['Input']
    y_val = npzfile['Output']
    y_val = np.ravel(y_val)
    x_val = x_val_all[:,:,index]
    if channel_first:
        x_val = x_val.transpose(0,2,1)
    
    npzfile = np.load(dir_dataset + 'TD.npz')
    x_test_all = npzfile['Input']
    y_test = npzfile['Output']
    y_test = np.ravel(y_test)
    x_test = x_test_all[:,:,index]
    if channel_first:
        x_test = x_test.transpose(0,2,1)
    
    if flatten == True:
        x_train = x_train.reshape([x_train.shape[0],-1])
        x_val = x_val.reshape([x_val.shape[0],-1])
        x_test = x_test.reshape([x_test.shape[0],-1])
    
    nb_classes = len(np.unique(np.concatenate((y_train, y_val, y_test), axis=0)))
    
    return x_train, y_train ,x_val, y_val, x_test, y_test, nb_classes

dataset = "archives/AQM/TDvsPC_new_no_shuffle/"

x_R, y_R ,x_L , y_L, x_TD, y_TD, nb_classes = load_dataset_v1(dataset,channel_first=True,flatten=False,shuffle=False,random_state=0)

X_d = np.concatenate([x_R,x_L,x_TD])
# X_d = X_d.reshape([len(X_d),-1])
y_d = np.concatenate([y_R,y_L,y_TD])
import  pandas as pd

R_data = pd.read_csv('DataPC - Right.csv', header=None)
R_idx = R_data[1].to_numpy()


L_data = pd.read_csv('DataPC - Left.csv', header=None)
L_idx = L_data[1].to_numpy()
L_idx = L_idx + R_idx[-1]

TD_data = pd.read_csv('DataTD - By Subject.csv', header=None)
TD_idx = TD_data[1].to_numpy()
TD_idx = TD_idx + L_idx[-1]

cycle_end_idx = np.concatenate([R_idx,L_idx,TD_idx])

patient_index_range = np.arange(len(R_idx)+len(L_idx)+len(TD_idx))
patient_class = np.concatenate([np.ones(len(R_idx)+len(L_idx)), np.zeros(len(TD_idx))])


x_train, x_test, y_train, y_test = train_test_split(patient_index_range, patient_class, test_size=0.4, stratify=patient_class,random_state=0)
x_val, x_test, y_val, y_test = train_test_split(x_test,y_test, test_size=0.75, random_state=0, stratify=y_test)
print(np.unique(y_train,return_counts=True))
print(np.unique(y_val,return_counts=True))
print(np.unique(y_test,return_counts=True))

(array([0., 1.]), array([35, 89]))
(array([0., 1.]), array([ 6, 14]))
(array([0., 1.]), array([18, 45]))


## Load function to evaluation

In [33]:
from torch.utils.data import Dataset
from utils.utils import gait_dataloader
from utils.utils import init_model
## lime
import torch
import pytorch_lightning as pl
class Dataset_torch(Dataset):

    def __init__(self, data,with_label=True):
        self.with_label =  with_label
        
        if self.with_label:
            self.data_x, self.data_y = data
        else:
            self.data_x = data
    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        if self.with_label:
            return self.data_x[idx], self.data_y[idx]
        else:
            return self.data_x[idx]


        
# batch_predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)],'new_train_lrR_ET_TDvsPC_new_inceptiom_full_InceptionTime/model-epoch=01-val_loss=0.74-val_accuracy=0.91.ckpt')

In [34]:
def train_model(D_name,Net_name):
    a_list = []
    for i in range(10):
        print("train number -", i)
        classifer = init_model(model_name=Net_name,lr_reduce=True, earlystopping=True, lr_patience=1, batch_size=32,lr=0.001, max_epochs=50,gpu=[0],default_root_dir="./new_train_lrR_ET_"+D_name+"_"+Net_name+"/")
        classifer.fit(X_d[patients_idx_to_cycles_idx(x_train,cycle_end_idx)], y_d[patients_idx_to_cycles_idx(x_train,cycle_end_idx)], X_d[patients_idx_to_cycles_idx(x_val,cycle_end_idx)], y_d[patients_idx_to_cycles_idx(x_val,cycle_end_idx)],ckpt_monitor='val_accuracy')
#         a_list.append(accuracy_score(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)], classifer.predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)])))
        print("train number end -", i)
#     list_f1 = []
#     list_accuracy = []
    
#     import os
#     filePath = "./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"
#     os.listdir(filePath)

#     j = 0
#     for i in os.listdir(filePath):
        
#         if 'model' in i:
#             j+=1
#             print('model_name_loading: ',i,j)
#             classifer = init_model(model_name=Net_name).load( "./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"+str(i))
#             print('compute acc',j)
#             list_accuracy.append(accuracy_score(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)], classifer.predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)])))
#             print('compute f1',j)
#             list_f1.append(f1_score(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)], classifer.predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)])))
#             print('return',j)
# #     return list_accuracy, list_f1 
#     return list_accuracy, "%.4f" %(np.mean(list_accuracy)) +"±"+ "%.4f" %(np.std(list_accuracy)),list_f1 , "%.4f" %(np.mean(list_f1)) +"±"+ "%.4f" %(np.std(list_f1))

In [37]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class Dataset_torch(Dataset):
    '''
    Adapt the dataset to feed to pytorch dataloader

    Parameter:
    data: input data to adapt
    with_label: True or False, with label or not

    Return prediction of model by confidence of each class
    '''
    def __init__(self, data,with_label=True):
        self.with_label =  with_label
        
        if self.with_label:
            self.data_x, self.data_y = data
        else:
            self.data_x = data
    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        if self.with_label:
            return self.data_x[idx], self.data_y[idx]
        else:
            return self.data_x[idx]
pred_set = Dataset_torch(X_d[:200],with_label=False)

In [38]:
# from utils.utils import *
# from classifiers.ResNet import *
# checkpoint = torch.load("/Volumes/Untitled/Desktop/Gait_DeepLearning_Diagnostic_Tool/model/new_train_lrR_ET_TDvsCPu_new_full_5test2025_ResNet/model-epoch=02-val_loss=0.24-val_accuracy=0.95.ckpt")
# model = ResNet(nb_classes=2)
# model.load_state_dict(checkpoint["state_dict"])
# trainer = pl.Trainer(accelerator='cpu')

# data_loader_pred = torch.utils.data.DataLoader(dataset=pred_set, batch_size=64,num_workers=0)

# pred = trainer.predict(model=model,dataloaders = data_loader_pred)

In [39]:
from sklearn.metrics import f1_score

def eval_model(D_name,Net_name):
    
    list_f1 = []
    list_accuracy = []

    import os
    filePath = "./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"
    os.listdir(filePath)

    for i in os.listdir(filePath):
        if 'model' in i:
            print(i)
            classifer = init_model(model_name=Net_name).load( "./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"+str(i))
            list_accuracy.append(accuracy_score(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)], classifer.predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)])))
            list_f1.append(f1_score(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)], classifer.predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)])))
    return list_accuracy, "%.4f" %(np.mean(list_accuracy)) +"±"+ "%.4f" %(np.std(list_accuracy)),list_f1 , "%.4f" %(np.mean(list_f1)) +"±"+ "%.4f" %(np.std(list_f1))

def batch_predict(images,model_path):
    pred_set = Dataset_torch(images,with_label=False)
    data_loader_pred = torch.utils.data.DataLoader(dataset=pred_set, batch_size=64,num_workers=0)
    
    
    classifer = init_model(model_name=Net_name).load(model_path)
    
    trainer = pl.Trainer(accelerator='cpu')
    pred = trainer.predict(model=classifer.model,dataloaders = data_loader_pred)
    pred = torch.cat(pred)
    return pred.detach().cpu().numpy()

from torch.utils.data import Dataset
from utils.utils import gait_dataloader
from utils.utils import init_model
## lime
import torch
import pytorch_lightning as pl
class Dataset_torch(Dataset):

    def __init__(self, data,with_label=True):
        self.with_label =  with_label
        
        if self.with_label:
            self.data_x, self.data_y = data
        else:
            self.data_x = data
    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        if self.with_label:
            return self.data_x[idx], self.data_y[idx]
        else:
            return self.data_x[idx]

def eval_model_5_bundle(D_name,Net_name,idx=None):
    
    list_f1 = []
    list_accuracy = []

    import os
    filePath = "./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"
    model_list = os.listdir(filePath)
    model_list.remove('logs')
    proba_sum_list = []
    if idx==None:
        for index in range(10):
            print(index)
            proba_sum = np.zeros([len(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]),2])
            for j in range(5):

                # print(model_list[index])
    #             classifer = init_model(model_name=Net_name).load( "./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"+model_list[index*5+j])
                print(batch_predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)],"./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"+model_list[index*5+j]))
                proba_sum +=batch_predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)],"./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"+model_list[index*5+j])/5
            proba_sum_list.append(proba_sum)
    else:
            index = idx
            print(index)
            proba_sum = np.zeros([len(y_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)]),2])
            for j in range(5):

                # print(model_list[index])
    #             classifer = init_model(model_name=Net_name).load( "./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"+model_list[index*5+j])
                print(batch_predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)],"./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"+model_list[index*5+j]))
                proba_sum +=batch_predict(X_d[patients_idx_to_cycles_idx(x_test,cycle_end_idx)],"./new_train_lrR_ET_"+D_name+"_"+Net_name+"/"+model_list[index*5+j])/5
            proba_sum_list = (proba_sum)
    return proba_sum_list

## Load the ranking of angles of different XAI methods

In [40]:
X_d_original = X_d.copy()
arr_all = np.load('./XAI_ranking/LIME_TDvsCPu_weight.npy')

## Evaluation of Fidelity by replace the last half to zero for ResNet

In [14]:
arr = arr_all[0]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,last_half_arr,:] = 0
print(X_d)
list_result_all = []

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['ResNet']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)
#0.65 0.35

path_img = './img_cm/'

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_ResNet/prediction_test.npy')

save = path_img+D_name_add+"_ResNet_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[ 3  6  0 14 17 19 20 11  9  2  8]
[10  7 21  5 13 16 15 12  1  4 18]
[[[  9.83950043   9.7201004    9.6335001  ...   9.47509956   9.24769974
     9.04039955]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -1.16919994  -0.93102998  -0.73694998 ...  -2.81369996  -2.54369998
    -2.28780007]
  ...
  [  6.59590006   6.85389996   6.91020012 ...   4.11780024   3.96300006
     3.72989988]
  [ -2.87540007  -3.09050012  -3.13870001 ...  -6.00860023  -6.19229984
    -6.19449997]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[  9.04039955   8.82820034   8.66639996 ...  13.4090004   13.08100033
    12.73579979]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -2.28780007  -2.0000999   -1.73239994 ...   0.29835001   0.66356999
     0.96999002]
  ...
  [  3.72989988   3.39369988   3.03649998 ...   6.45079994   6.55550003
     6.6479001 ]
  [ -6.19449997  -5.98360014  -5.56080008 ...  -0.

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


Predicting DataLoader 0:  25%|████▊              | 4/16 [00:00<00:00, 17.01it/s]

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


...


[[0.08687324 0.91312677]
 [0.24339058 0.75660944]
 [0.14013036 0.85986966]
 ...
 [0.00227132 0.99772865]
 [0.00124834 0.99875164]
 [0.00250565 0.99749434]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:00<00:00, 18.11it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:00<00:00, 18.12it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



[[0.08056045 0.91943955]
 [0.11433894 0.88566107]
 [0.09133916 0.9086608 ]
 ...
 [0.00146313 0.9985368 ]
 [0.00157155 0.99842846]
 [0.00191082 0.99808913]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:00<00:00, 18.26it/s]
cycles
Accuracy:
Accuracy_list: [0.7796442687747036, 0.8932806324110671, 0.8320158102766798, 0.8893280632411067, 0.8507905138339921, 0.8814229249011858, 0.8399209486166008, 0.8586956521739131, 0.8784584980237155, 0.8695652173913043]
Accuracy_mean: 0.8573122529644269
Accuracy_std: 0.03242913792966476
---
Confusion matrix
Confusion matrix mean: [[187.   12. ]
 [132.4 680.6]]
Confusion matrix std: [[ 5.03984127  5.03984127]
 [32.444414   32.444414  ]]
---
F1
F1_list: [0.8437281009110021, 0.9292267365661862, 0.8849797023004059, 0.9273670557717251, 0.8980418636056718, 0.9215686274509803, 0.8922872340425532, 0.9058591178406846, 0.9194499017681729, 0.9132720105124835]
F1_mean: 0.9035780350769868
F1_std: 0.024463383744988514
---
Sensitivity
Sensitivity list: [

## Evaluation of Fidelity by replace the fast half to zero for ResNet

In [15]:
X_d = X_d_original.copy()
X_d[:,first_half_arr,:] = 0
print(X_d)

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['ResNet']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)
#0.65 0.35

path_img = './img_cm/'

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_ResNet/prediction_test.npy')

save = path_img+D_name_add+"_ResNet_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[[[  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -0.09395     -0.20055     -0.26530001 ...  -1.81280005  -1.99220002
    -2.14240003]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -1.2658      -0.74980998  -0.40599999 ...  -4.43109989  -4.59639978
    -4.76749992]]

 [[  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -2.14240003  -2.2730999   -2.33719993 ...   0.41102999   0.15759
    -0.10672   ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -4.76749992  -4.95900011  -5.13100004 ...  -9.70460033  -8.66300011
  

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


Predicting DataLoader 0:  19%|███▌               | 3/16 [00:00<00:00, 17.84it/s]

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


...


[[1.0000000e+00 3.9297330e-09]
 [1.0000000e+00 3.2229184e-09]
 [1.0000000e+00 3.7789709e-09]
 ...
 [1.0000000e+00 2.6146342e-08]
 [9.9999750e-01 2.5194629e-06]
 [9.9999905e-01 9.1893264e-07]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:00<00:00, 18.04it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:00<00:00, 18.23it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



[[1.0000000e+00 8.0815410e-11]
 [1.0000000e+00 1.3657364e-10]
 [1.0000000e+00 9.7587285e-11]
 ...
 [1.0000000e+00 9.6169952e-09]
 [9.9999595e-01 4.0922087e-06]
 [9.9999845e-01 1.6066799e-06]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:00<00:00, 18.14it/s]
cycles
Accuracy:
Accuracy_list: [0.2134387351778656, 0.20355731225296442, 0.19268774703557312, 0.19960474308300397, 0.19861660079051383, 0.20553359683794467, 0.20059288537549408, 0.1966403162055336, 0.1976284584980237, 0.1808300395256917]
Accuracy_mean: 0.19891304347826086
Accuracy_std: 0.008076816635146021
---
Confusion matrix
Confusion matrix mean: [[182.2  16.8]
 [793.9  19.1]]
Confusion matrix std: [[4.57820926 4.57820926]
 [8.56095789 8.56095789]]
---
F1
F1_list: [0.06572769953051644, 0.06496519721577726, 0.01684717208182912, 0.03341288782816229, 0.05367561260210035, 0.06944444444444445, 0.05380116959064327, 0.02867383512544803, 0.04918032786885246, 0.011918951132300357]
F1_mean: 0.044764729742007396
F1_std: 0.0

## Evaluation of Fidelity by replace the last half to zero for LSTM

In [16]:
X_d = X_d_original.copy()
arr = arr_all[1]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,last_half_arr,:] = 0
print(X_d)
path_img = './img_cm/'

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['LSTM']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)
#0.65 0.35

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_LSTM/prediction_test.npy')

save = path_img+D_name_add+"_LSTM_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

[ 3  6  9 11 17 19 14 13 10  5  0]
[15 16  8  2 21  1  7 12 18 20  4]
[[[  9.83950043   9.7201004    9.6335001  ...   9.47509956   9.24769974
     9.04039955]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  ...
  [  6.59590006   6.85389996   6.91020012 ...   4.11780024   3.96300006
     3.72989988]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[  9.04039955   8.82820034   8.66639996 ...  13.4090004   13.08100033
    12.73579979]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  ...
  [  3.72989988   3.39369988   3.03649998 ...   6.45079994   6.55550003
     6.6479001 ]
  [  0.           0.           0.         ...   0.           0.
     0.   

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


Predicting DataLoader 0:  12%|██▍                | 2/16 [00:00<00:02,  5.70it/s]

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:02<00:00,  5.55it/s]
[[5.1726136e-02 9.4827390e-01]
 [3.2530971e-02 9.6746910e-01]
 [1.9797318e-02 9.8020267e-01]
 ...
 [1.1409743e-03 9.9885905e-01]
 [5.5397325e-04 9.9944597e-01]
 [6.7040097e-04 9.9932957e-01]]


...

Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:03<00:00,  4.26it/s]


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:03<00:00,  4.20it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



[[2.0150816e-02 9.7984916e-01]
 [7.6424875e-03 9.9235743e-01]
 [3.1808398e-03 9.9681914e-01]
 ...
 [2.6726022e-05 9.9997330e-01]
 [1.2211668e-05 9.9998784e-01]
 [1.4186141e-05 9.9998581e-01]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:03<00:00,  4.13it/s]
cycles
Accuracy:
Accuracy_list: [0.9100790513833992, 0.9100790513833992, 0.9090909090909091, 0.9061264822134387, 0.9090909090909091, 0.9090909090909091, 0.8952569169960475, 0.9051383399209486, 0.908102766798419, 0.8843873517786561]
Accuracy_mean: 0.9046442687747035
Accuracy_std: 0.007945180208587015
---
Confusion matrix
Confusion matrix mean: [[172.1  26.9]
 [ 69.6 743.4]]
Confusion matrix std: [[3.3        3.3       ]
 [9.93176721 9.93176721]]
---
F1
F1_list: [0.9425142135186355, 0.9432314410480349, 0.9422835633626098, 0.9400630914826499, 0.9420654911838791, 0.9420654911838791, 0.9321382842509603, 0.9392405063291139, 0.9414726242920075, 0.9250480461242793]
F1_mean: 0.9390122752776049
F1_std: 0.005557621627831604
---

## Evaluation of Fidelity by replace the first half to zero for LSTM

In [17]:
X_d = X_d_original.copy()
arr = arr_all[1]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,first_half_arr,:] = 0
print(X_d)
path_img = './img_cm/'

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['LSTM']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_LSTM/prediction_test.npy')

save = path_img+D_name_add+"_LSTM_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

[ 3  6  9 11 17 19 14 13 10  5  0]
[15 16  8  2 21  1  7 12 18 20  4]
[[[  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -0.09395     -0.20055     -0.26530001 ...  -1.81280005  -1.99220002
    -2.14240003]
  [ -1.16919994  -0.93102998  -0.73694998 ...  -2.81369996  -2.54369998
    -2.28780007]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -2.87540007  -3.09050012  -3.13870001 ...  -6.00860023  -6.19229984
    -6.19449997]
  [ -1.2658      -0.74980998  -0.40599999 ...  -4.43109989  -4.59639978
    -4.76749992]]

 [[  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -2.14240003  -2.2730999   -2.33719993 ...   0.41102999   0.15759
    -0.10672   ]
  [ -2.28780007  -2.0000999   -1.73239994 ...   0.29835001   0.66356999
     0.96999002]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -6.19449997  -5.98360014  -5.56080008 ...  -0.67334002  -

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   6%|█▏                 | 1/16 [00:00<00:03,  3.96it/s]

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:03<00:00,  4.06it/s]
[[0.13429356 0.86570644]
 [0.0883727  0.91162735]
 [0.0956146  0.90438545]
 ...
 [0.18419257 0.81580746]
 [0.28992254 0.71007746]
 [0.3474043  0.65259564]]


...


[[0.9933014  0.00669865]
 [0.9801522  0.01984784]
 [0.5285062  0.47149375]
 ...
 [0.05839445 0.9416055 ]
 [0.00936277 0.99063724]
 [0.0107504  0.9892496 ]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:03<00:00,  4.13it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:03<00:00,  4.09it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



[[9.99374211e-01 6.25823159e-04]
 [9.99971628e-01 2.83811478e-05]
 [9.99898434e-01 1.01594575e-04]
 ...
 [9.99964952e-01 3.50800801e-05]
 [9.97234046e-01 2.76601431e-03]
 [9.98210788e-01 1.78917614e-03]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:04<00:00,  3.94it/s]
cycles
Accuracy:
Accuracy_list: [0.6936758893280632, 0.5, 0.4851778656126482, 0.6393280632411067, 0.5731225296442688, 0.6116600790513834, 0.682806324110672, 0.6729249011857708, 0.5820158102766798, 0.5503952569169961]
Accuracy_mean: 0.5991106719367589
Accuracy_std: 0.07022494449919163
---
Confusion matrix
Confusion matrix mean: [[162.   37. ]
 [368.7 444.3]]
Confusion matrix std: [[16.36459593 16.36459593]
 [82.53247846 82.53247846]]
---
F1
F1_list: [0.7776183644189383, 0.5697278911564626, 0.5481352992194276, 0.7290274684484038, 0.6544, 0.6965250965250965, 0.7688984881209503, 0.7731322823851954, 0.6629482071713148, 0.6285714285714286]
F1_mean: 0.6808984526017218
F1_std: 0.07868536489700588
---
Sensitivity


## Evaluation of Fidelity by replace the last half to zero for InceptionTime

In [18]:
X_d = X_d_original.copy()

arr = arr_all[2]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,last_half_arr,:] = 0
print(X_d)
path_img = './img_cm/'

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['InceptionTime']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_InceptionTime/prediction_test.npy')

save = path_img+D_name_add+"_InceptionTime_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


[ 3 17  0  6 11  8 20  9  2 19 10]
[13 14  7 21  5 16 12  1 18  4 15]
[[[  9.83950043   9.7201004    9.6335001  ...   9.47509956   9.24769974
     9.04039955]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -1.16919994  -0.93102998  -0.73694998 ...  -2.81369996  -2.54369998
    -2.28780007]
  ...
  [  6.59590006   6.85389996   6.91020012 ...   4.11780024   3.96300006
     3.72989988]
  [ -2.87540007  -3.09050012  -3.13870001 ...  -6.00860023  -6.19229984
    -6.19449997]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[  9.04039955   8.82820034   8.66639996 ...  13.4090004   13.08100033
    12.73579979]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -2.28780007  -2.0000999   -1.73239994 ...   0.29835001   0.66356999
     0.96999002]
  ...
  [  3.72989988   3.39369988   3.03649998 ...   6.45079994   6.55550003
     6.6479001 ]
  [ -6.19449997  -5.98360014  -5.56080008 ...  -0.

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  3.03it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...


[[0.01624322 0.9837568 ]
 [0.08099665 0.9190033 ]
 [0.01401749 0.9859825 ]
 ...
 [0.2351152  0.7648848 ]
 [0.03184614 0.9681539 ]
 [0.24339785 0.75660217]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  2.95it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  2.92it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



[[1.1243429e-03 9.9887568e-01]
 [6.2583615e-03 9.9374157e-01]
 [4.4392473e-03 9.9556077e-01]
 ...
 [8.6281756e-03 9.9137187e-01]
 [8.4736629e-04 9.9915266e-01]
 [6.3404478e-03 9.9365956e-01]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  2.93it/s]
cycles
Accuracy:
Accuracy_list: [0.883399209486166, 0.8231225296442688, 0.849802371541502, 0.7598814229249012, 0.8023715415019763, 0.7509881422924901, 0.8300395256916996, 0.7154150197628458, 0.7519762845849802, 0.7450592885375494]
Accuracy_mean: 0.791205533596838
Accuracy_std: 0.051571277833106256
---
Confusion matrix
Confusion matrix mean: [[185.2  13.8]
 [197.5 615.5]]
Confusion matrix std: [[ 5.43691089  5.43691089]
 [55.00772673 55.00772673]]
---
F1
F1_list: [0.9241645244215938, 0.8797850906648758, 0.8986666666666666, 0.8270462633451957, 0.861878453038674, 0.8194842406876791, 0.8834688346883469, 0.7900874635568513, 0.8213523131672598, 0.8125]
F1_mean: 0.8518433850237143
F1_std: 0.04154249654067003
---
Sensitivity

## Evaluation of Fidelity by replace the first half to zero for InceptionTime

In [19]:
X_d = X_d_original.copy()
arr = arr_all[2]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,first_half_arr,:] = 0
print(X_d)
path_img = './img_cm/'

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['InceptionTime']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_InceptionTime/prediction_test.npy')

save = path_img+D_name_add+"_InceptionTime_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[ 3 17  0  6 11  8 20  9  2 19 10]
[13 14  7 21  5 16 12  1 18  4 15]
[[[  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -0.09395     -0.20055     -0.26530001 ...  -1.81280005  -1.99220002
    -2.14240003]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -1.2658      -0.74980998  -0.40599999 ...  -4.43109989  -4.59639978
    -4.76749992]]

 [[  0.           0.           0.         ...   0.           0.
     0.        ]
  [ -2.14240003  -2.2730999   -2.33719993 ...   0.41102999   0.15759
    -0.10672   ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [ 

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   6%|█▏                 | 1/16 [00:00<00:05,  2.86it/s]

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


...


[[9.9952888e-01 4.7112632e-04]
 [9.9981910e-01 1.8098479e-04]
 [9.9996984e-01 3.0204030e-05]
 ...
 [9.9998546e-01 1.4525016e-05]
 [9.9966681e-01 3.3317425e-04]
 [9.9940038e-01 5.9963617e-04]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  2.98it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  3.00it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



[[9.9995124e-01 4.8749436e-05]
 [9.9995852e-01 4.1453426e-05]
 [9.9999332e-01 6.6878611e-06]
 ...
 [9.9987030e-01 1.2971912e-04]
 [9.9870074e-01 1.2992009e-03]
 [9.9918598e-01 8.1404508e-04]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  2.99it/s]
cycles
Accuracy:
Accuracy_list: [0.22826086956521738, 0.2391304347826087, 0.20059288537549408, 0.1956521739130435, 0.24011857707509882, 0.20948616600790515, 0.208498023715415, 0.18873517786561264, 0.18774703557312253, 0.1907114624505929]
Accuracy_mean: 0.20889328063241108
Accuracy_std: 0.01921777989113552
---
Confusion matrix
Confusion matrix mean: [[187.   12. ]
 [788.6  24.4]]
Confusion matrix std: [[ 3.          3.        ]
 [21.17640196 21.17640196]]
---
F1
F1_list: [0.09919261822376009, 0.14253897550111358, 0.03575685339690107, 0.023980815347721823, 0.12514220705346984, 0.05660377358490566, 0.05430932703659976, 0.007255139056831923, 0.004842615012106538, 0.012062726176115802]
F1_mean: 0.05616850503895262
F1_std:

In [20]:
df_test = pd.DataFrame (list_result_all)
df_test.T

,0,1,2,3,4,5
0,0.857312±0.032429,0.198913±0.008077,0.904644±0.007945,0.599111±0.070225,0.791206±0.051571,0.208893±0.019218
1,0.837146±0.039907,0.023493±0.010530,0.914391±0.012216,0.546494±0.101516,0.757073±0.067660,0.030012±0.026047
2,0.939698±0.025326,0.915578±0.023006,0.864824±0.016583,0.814070±0.082234,0.930653±0.027321,0.939698±0.015075
3,0.903578±0.024463,0.044765±0.019706,0.939012±0.005558,0.680898±0.078685,0.851843±0.041542,0.056169±0.047431
4,0.888422±0.023600,0.469536±0.010846,0.889608±0.006059,0.680282±0.033645,0.843863±0.028613,0.484855±0.010200


In [41]:


arr_all = np.load('./XAI_ranking/DeepShap_TDvsCPu_weight.npy')

In [42]:
X_d = X_d_original.copy()
arr = arr_all[0]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,last_half_arr,:] = 0
print(X_d)
list_result_all = []

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['ResNet']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)
#0.65 0.35

path_img = './img_cm/'

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_ResNet/prediction_test.npy')

save = path_img+D_name_add+"_ResNet_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

X_d = X_d_original.copy()
X_d[:,first_half_arr,:] = 0
print(X_d)

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['ResNet']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)
#0.65 0.35

path_img = './img_cm/'

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_ResNet/prediction_test.npy')

save = path_img+D_name_add+"_ResNet_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))


X_d = X_d_original.copy()
arr = arr_all[1]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,last_half_arr,:] = 0
print(X_d)
path_img = './img_cm/'

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['LSTM']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)
#0.65 0.35

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_LSTM/prediction_test.npy')

save = path_img+D_name_add+"_LSTM_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

X_d = X_d_original.copy()
arr = arr_all[1]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,first_half_arr,:] = 0
print(X_d)
path_img = './img_cm/'

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['LSTM']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_LSTM/prediction_test.npy')

save = path_img+D_name_add+"_LSTM_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

X_d = X_d_original.copy()

arr = arr_all[2]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,last_half_arr,:] = 0
print(X_d)
path_img = './img_cm/'

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['InceptionTime']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_InceptionTime/prediction_test.npy')

save = path_img+D_name_add+"_InceptionTime_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

X_d = X_d_original.copy()
arr = arr_all[2]
first_half_arr = arr[:len(arr) // 2]
last_half_arr = arr[len(arr) // 2:]
print(first_half_arr)
print(last_half_arr)

X_d[:,first_half_arr,:] = 0
print(X_d)
path_img = './img_cm/'

D_name_add = 'TDvsCPu'
D_name = D_name_add+"_full_5_original_weight"
# Net_name_list = ['ResNet','LSTM','InceptionTime']
Net_name_list = ['InceptionTime']
for Net_name in Net_name_list:
    # for j in range(5):
        # train_model(D_name=D_name,Net_name=Net_name)
    out = eval_model_5_bundle(D_name,Net_name)
    np.save( './new_train_lrR_ET_'+D_name+'_'+Net_name+'/prediction_test.npy' ,out)

out = np.load( './new_train_lrR_ET_'+D_name_add+'_full_5_original_weight_InceptionTime/prediction_test.npy')

save = path_img+D_name_add+"_InceptionTime_original_weight"
print("cycles")
results_cycles = print_result_bundle_i_cycles(out,y_d,x_test,cycle_end_idx,label_list=['TD', 'CPu'],n_model=10)
list_result_all.append(output_csv(results_cycles))

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


[ 3 14  6 17 11 10  0 20  9 13 19]
[21  2  5  8 16  7  1 15  4 18 12]
[[[  9.83950043   9.7201004    9.6335001  ...   9.47509956   9.24769974
     9.04039955]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  ...
  [  6.59590006   6.85389996   6.91020012 ...   4.11780024   3.96300006
     3.72989988]
  [ -2.87540007  -3.09050012  -3.13870001 ...  -6.00860023  -6.19229984
    -6.19449997]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[  9.04039955   8.82820034   8.66639996 ...  13.4090004   13.08100033
    12.73579979]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  ...
  [  3.72989988   3.39369988   3.03649998 ...   6.45079994   6.55550003
     6.6479001 ]
  [ -6.19449997  -5.98360014  -5.56080008 ...  -0.67334002  -1.161

/opt/anaconda3/envs/env_gait_new/lib/python3.13/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:00<00:00, 16.98it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...


[[0.9973564  0.00264361]
 [0.9933134  0.00668661]
 [0.9750437  0.02495626]
 ...
 [0.9961287  0.00387127]
 [0.84213585 0.15786415]
 [0.94795895 0.05204111]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  2.86it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  2.87it/s]

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



[[9.9993241e-01 6.7630230e-05]
 [9.9979407e-01 2.0591254e-04]
 [9.9846703e-01 1.5329330e-03]
 ...
 [9.9822479e-01 1.7751625e-03]
 [9.7187042e-01 2.8129634e-02]
 [9.8840630e-01 1.1593734e-02]]
Predicting DataLoader 0: 100%|██████████████████| 16/16 [00:05<00:00,  2.87it/s]
cycles
Accuracy:
Accuracy_list: [0.2341897233201581, 0.2766798418972332, 0.22924901185770752, 0.1966403162055336, 0.23221343873517786, 0.2341897233201581, 0.233201581027668, 0.21146245059288538, 0.20355731225296442, 0.1976284584980237]
Accuracy_mean: 0.22490118577075102
Accuracy_std: 0.022745309907310766
---
Confusion matrix
Confusion matrix mean: [[197.    2. ]
 [782.4  30.6]]
Confusion matrix std: [[ 2.44948974  2.44948974]
 [23.99249883 23.99249883]]
---
F1
F1_list: [0.08930669800235018, 0.18666666666666668, 0.0951276102088167, 0.0, 0.0912280701754386, 0.08930669800235018, 0.09345794392523364, 0.04316546762589928, 0.01707317073170732, 0.002457002457002457]
F1_mean: 0.0707789327795465
F1_std: 0.053820295164995625
-

## ALL results will be in this table

In [43]:
df_test = pd.DataFrame (list_result_all)
df_test.T

,0,1,2,3,4,5
0,0.782609±0.034992,0.213043±0.024334,0.913538±0.002906,0.793083±0.028203,0.816403±0.039213,0.224901±0.022745
1,0.755351±0.043301,0.031611±0.027756,0.917835±0.007853,0.868020±0.048087,0.804797±0.052382,0.037638±0.029511
2,0.893970±0.001508,0.954271±0.021373,0.895980±0.019076,0.486935±0.089877,0.863819±0.021491,0.989950±0.012309
3,0.847395±0.028647,0.059383±0.050811,0.944604±0.002185,0.870203±0.021222,0.874739±0.030684,0.070779±0.053820
4,0.824660±0.022179,0.492941±0.020715,0.906908±0.005874,0.677477±0.031627,0.834308±0.020102,0.513794±0.013250
